import

In [1]:
# === Cell 1: Imports ===

import warnings
warnings.filterwarnings("ignore")

from sentence_transformers import SentenceTransformer
import hopsworks
import config


In [ ]:
import os
# os.environ["HF_API_KEY"] = ""
os.environ["SILICONFLOW_API_KEY"] = ""

In [3]:
# === Cell 2: Connect to Hopsworks ===

from config import HOPSWORKS_API_KEY
# project = hopsworks.login()

project = hopsworks.login(
        # project=HOPSWORKS_PROJECT,
        api_key_value=HOPSWORKS_API_KEY
    )
fs = project.get_feature_store()


2026-01-11 03:18:08,173 INFO: Initializing external client
2026-01-11 03:18:08,176 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-11 03:18:09,979 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1286333


In [4]:
# === Cell 3: Load Feature Views ===

metadata_fv = fs.get_feature_view(
    name="paper_metadata_fv",
    version=1,
)

chunk_fv = fs.get_feature_view(
    name="paper_chunk_fv",
    version=1,
)

metadata_fv.init_serving(1)
chunk_fv.init_serving(1)


In [5]:
# === Cell 4: Load Embedding Model ===
sentence_transformer = SentenceTransformer(
    config.EMBEDDING_MODEL_NAME
)

2026-01-11 03:18:21,014 INFO: Use pytorch device_name: cpu
2026-01-11 03:18:21,015 INFO: Load pretrained SentenceTransformer: all-MiniLM-L6-v2


In [6]:
# === Cell 5: Similarity Search Engine ===

from functions.similarity_search import SimilaritySearchEngine

search_engine = SimilaritySearchEngine(
    embedding_model=sentence_transformer,
    metadata_feature_view=metadata_fv,
    chunk_feature_view=chunk_fv,
)


In [7]:
# === Cell 6: Context Builder ===

from functions.context_builder import ContextBuilder

context_builder = ContextBuilder(
    max_tokens=2000,
    max_chunks=8,
)


In [8]:
# === Cell 7: Prompt Synthesizer ===

from functions.prompt_synthesis import PromptSynthesizer
prompt_synthesizer = PromptSynthesizer()

# from functions.prompt_synthesis_debug import DebugPromptSynthesizer

# prompt_synthesizer = DebugPromptSynthesizer()



In [9]:
# === Cell 8: MCP Dispatcher ===

from functions.mcp_dispatcher import MCPDispatcher

mcp_dispatcher = MCPDispatcher(
    search_engine=search_engine
)


In [10]:
# === Cell 9: Agentic Inference ===

from functions.agent_loop import AgenticInference
from functions.llm_wrapper import LLMWrapper  

# llm = LLMWrapper(
#     model_name_or_path="mistralai/Mistral-7B-v0.1"  
# )# local llm
llm = LLMWrapper(
    model="Qwen/Qwen3-8B",
    base_url="http://api.siliconflow.cn/v1/",
    api_key=os.getenv("SILICONFLOW_API_KEY"),
    temperature=0.2,
    max_tokens=1024,
)

agent = AgenticInference(
    llm=llm,
    search_engine=search_engine,
    context_builder=context_builder,
    prompt_synthesizer=prompt_synthesizer,
    mcp_dispatcher=mcp_dispatcher,
)


In [11]:
# === Cell 10: Run Agent ===

from functions.agent_ui import launch_agent_ui

ui = launch_agent_ui(agent)
ui.launch(inline=True)


* Running on local URL:  http://127.0.0.1:7860
2026-01-11 03:18:26,984 INFO: HTTP Request: GET http://127.0.0.1:7860/gradio_api/startup-events "HTTP/1.1 200 OK"
2026-01-11 03:18:27,006 INFO: HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"
* To create a public link, set `share=True` in `launch()`.


2026-01-11 03:18:27,298 INFO: HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
2026-01-11 03:19:00,156 WARNING: DeprecationWarning: 'HTTP_422_UNPROCESSABLE_ENTITY' is deprecated. Use 'HTTP_422_UNPROCESSABLE_CONTENT' instead.



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

=== CONTEXT BUNDLE ===
{'items': [{'source_id': '6J5WS7DJ#chunk-1', 'paper_id': '6J5WS7DJ', 'content': 'Computers in Biology and Medicine 195 (2025) 110623 11 Lastly, we experimented with Controllable GAN [45], an extension of the traditional conditional GAN that can generate samples with specified features in the form of labels. Like MGAN, Controllable GAN also in\xad troduces a classifier in the adversarial game [45]. describes the training algorithm and objective functions for Controllable GAN. Since we were interested in generating synthetic samples for one CAD class and did not possess labels for specific features, we first clustered our CAD heart sound recordings with the K-means clustering algorithm. Acoustic fea\xad tures used for clustering included the mean and standard deviation of the Mel Frequency Cepstral Coefficients (MFCCs) across all frames in the audio and the rate of change of the MFCCs between successive frames. The elbow curve method was used to estimate the optima

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

=== CONTEXT BUNDLE ===
{'items': [{'source_id': 'PDU98SKK#chunk-2', 'paper_id': 'PDU98SKK', 'content': 'c model, a class of generative models that has attracted a lot of attention lately. Our proposal is efficient, flexible, and lightweight by using powerful DiffWave architecture as a baseline model. This approach has revealed the capability to generate long ECG signals similar to those obtained from patients. The Physionet dataset, including MIT-BIH Arrhythmia and MIT-BIH Atrial Fibrillation Databases, is applied to train and test our models. Consequently, we produced 10-second ECGs with several rhythms that were not visible in earlier investigations. The proposal outperforms the most satisfactory research in ECGs when comparing our high-quality synthesized data with actual signals on DNN models. Furthermore, our method also offers a novel technique to investigate electrocardiogram data without using an ECG simulator. CCS CONCEPTS • Applied computing →Life and medical sciences; Bioinf